In [1]:
import numpy as np
import pandas as pd

# Interacting with Databases 

En un entorno empresarial, la mayoría de los datos no pueden almacenarse en archivos de texto o Excel. Las bases de datos relacionales basadas en SQL (como SQL Server, PostgreSQL y MySQL) se usan ampliamente, y muchas bases de datos alternativas se han vuelto bastante populares. La elección de la base de datos generalmente depende del rendimiento, la integridad de los datos y las necesidades de escalabilidad de una aplicación.

Cargar datos de SQL en un DataFrame es bastante sencillo, y pandas tiene algunas funciones para simplificar el proceso. Como ejemplo, se crea una base de datos SQLite usando el controlador sqlite3 incorporado de Python

In [2]:
import sqlite3

In [3]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""


In [4]:
con = sqlite3.connect('mydata.sqlite')

In [5]:
con.execute(query)

OperationalError: table test already exists

In [6]:
con.commit()

Luego, inserte algunas filas de datos

In [7]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [8]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [9]:
con.executemany(stmt, data)

In [10]:
con.commit()

La mayoría de los controladores Python SQL (PyODBC, psycopg2, MySQLdb, pymssql, etc.) devuelven una lista de tuplas al seleccionar datos de una tabla

In [11]:
cursor = con.execute('select * from test')

In [12]:
rows = cursor.fetchall()

In [13]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Puede pasar la lista de tuplas al constructor de DataFrame, pero también necesita los nombres de columna, contenidos en el atributo de descripción del cursor

In [14]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [15]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5


Esto es un poco molesto que prefiere no repetir cada vez que consulta la base de datos. El proyecto SQLAlchemy es un popular kit de herramientas Python SQL que abstrae muchas de las diferencias comunes entre las bases de datos SQL. pandas tiene una función read_sql que le permite leer datos fácilmente desde una conexión SQLAlchemy general. Aquí, nos conectaremos a la misma base de datos SQLite con SQLAlchemy y leeremos los datos de la tabla creada antes

In [16]:
import sqlalchemy as sqla

In [17]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [18]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
